In [1]:
import numpy as np
import sklearn.datasets
import tensorflow as tf

In [2]:
from sklearn.metrics import accuracy_score 
from tensorflow.keras.losses import mean_squared_error

In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

In [6]:
x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000,784)
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [7]:
import pandas as pd

In [8]:
y_train1 = pd.DataFrame(y_train).astype('str')
y_train1 = pd.get_dummies(y_train1)

In [9]:
y_train1 = y_train1.to_numpy(int)

In [15]:
def ReLU(x):
    return np.maximum(0,x)

def ReLU_derivative(x):
    return np.where(x>0,1,0)

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [16]:
# def gradientclip(x):
    # return np.where(np.where(x<-10,1,x)>10,1,x)

In [17]:
class NeuralNetwork:
    def __init__(self,input_size,hidden1_Size,hidden2_size,hidden3_size,output_size):
        #(hidden_layer1)
        self.w1= np.random.random((input_size,hidden1_Size)) *0.1 - 0.05
        self.b1= np.zeros((hidden1_Size)) 

        #(hidden layer2)      
        self.w2=np.random.random((hidden1_Size,hidden2_size)) *0.1 - 0.05
        self.b2=np.zeros((hidden2_size))

        #(hidden layer3)
        self.w3 =np.random.random((hidden2_size,hidden3_size)) *0.1 - 0.05
        self.b3 =np.zeros((hidden3_size))

        #(output_layer)
        self.w4 =np.random.random((hidden3_size,output_size)) *0.1 - 0.05
        self.b4 =np.zeros((output_size))

    
    def forward(self,x):
        self.x1=np.dot(x,self.w1) + self.b1
        self.y1=ReLU(self.x1) #output of hidden_layer1
        self.x2=np.dot(self.y1,self.w2) + self.b2
        self.y2=ReLU(self.x2)
        self.x3=np.dot(self.y2,self.w3) + self.b3
        self.y3=ReLU(self.x3)
        self.x4=np.dot(self.y3,self.w4) + self.b4
        self.o=tf.nn.softmax(self.x4)
        #print(self.o.numpy())

        return self.o.numpy()

    def backward(self,o,x,y,alpha):
        output_delta = (y-o) * 0.01
        #print(output_delta)
        hidden3_error = output_delta.dot(self.w4.T)
        hidden3_delta=hidden3_error*(ReLU_derivative(self.y3))

        hidden2_error = hidden3_delta.dot(self.w3.T)
        hidden2_delta= hidden2_error*(ReLU_derivative(self.y2))
        
        hidden1_error = hidden2_delta.dot(self.w2.T)
        hidden1_delta = hidden1_error*(ReLU_derivative(self.y1))

        self.w4+=(self.y3.T.dot(output_delta)*alpha)
        self.w3+=(self.y2.T.dot(hidden3_delta)*alpha)
        self.w2+=(self.y1.T.dot(hidden2_delta)*alpha)
        self.w1+=(x.reshape(1,784).T.dot(hidden1_delta)*alpha)

        self.b4+=(np.sum(output_delta,0)*alpha)/y.shape[0]
        self.b3+=(np.sum(hidden3_delta,0)*alpha)/y.shape[0]
        self.b2+=(np.sum(hidden2_delta,0)*alpha)/y.shape[0]
        self.b1+=(np.sum(hidden1_delta,0)*alpha)/y.shape[0]

        #print('bias:',self.y1.T.dot(hidden2_delta)*alpha)

    def train(self,x,y_t,y,alpha,epoch):
        LOSS=np.zeros(epoch,'float64')
        ACCURACY=np.zeros(epoch,'float64')
        for j in range(epoch):
            for i in range(len(x)):
                o = self.forward(x[i].reshape(1,784))
                self.backward(o,x[i],y_t[i].reshape(1,10),alpha)
            
            ot=self.forward(x)
            LOSS[j]=np.sum(mean_squared_error(y_t,ot))
            ACCURACY[j]=accuracy_score(y,np.argmax(ot,axis=1))
            print('epoch:',j,'loss:',LOSS[j],'accuracy:',ACCURACY[j])
    
    def predict(self,x):
        o = self.forward(x)
        return np.argmax(o,axis=1)

   
        


In [18]:
nn=NeuralNetwork(784,32,25,16,10)

In [19]:
nn.train(x_train,y_train1,y_train,0.05,10)

epoch: 0 loss: 5397.3555811991655 accuracy: 0.11236666666666667
epoch: 1 loss: 4966.007343497299 accuracy: 0.22228333333333333
epoch: 2 loss: 1477.4742776024614 accuracy: 0.8460166666666666
epoch: 3 loss: 807.9965022513396 accuracy: 0.9110166666666667
epoch: 4 loss: 599.4303395068055 accuracy: 0.9342333333333334
epoch: 5 loss: 511.1068706063887 accuracy: 0.9438666666666666
epoch: 6 loss: 441.9768046088677 accuracy: 0.9516166666666667
epoch: 7 loss: 388.0824606933768 accuracy: 0.9579166666666666
epoch: 8 loss: 344.11265356548654 accuracy: 0.9629666666666666
epoch: 9 loss: 309.815950035895 accuracy: 0.9665833333333333


In [20]:
nn.train(x_train,y_train1,y_train,0.1,20)

epoch: 0 loss: 328.6657575136547 accuracy: 0.9642666666666667
epoch: 1 loss: 311.0424989611912 accuracy: 0.9655833333333333
epoch: 2 loss: 283.1936567612685 accuracy: 0.9688166666666667
epoch: 3 loss: 249.9099876570859 accuracy: 0.9718333333333333
epoch: 4 loss: 255.93865146365383 accuracy: 0.97155
epoch: 5 loss: 231.250951500028 accuracy: 0.9742833333333333
epoch: 6 loss: 228.02241584171603 accuracy: 0.9748
epoch: 7 loss: 228.33881267401054 accuracy: 0.9747166666666667
epoch: 8 loss: 254.69440796121458 accuracy: 0.9723333333333334
epoch: 9 loss: 198.6622333166239 accuracy: 0.9779333333333333
epoch: 10 loss: 202.2657770423403 accuracy: 0.9776166666666667
epoch: 11 loss: 181.6412207620764 accuracy: 0.9804333333333334
epoch: 12 loss: 175.3607047467918 accuracy: 0.9808166666666667
epoch: 13 loss: 190.9336170434245 accuracy: 0.97935
epoch: 14 loss: 152.55330482672196 accuracy: 0.9832666666666666
epoch: 15 loss: 157.90026882852504 accuracy: 0.98285
epoch: 16 loss: 143.3379279780698 accuracy

In [21]:
accuracy_score(y_test,nn.predict(x_test))

0.9573

In [22]:
nn.predict(x_test)

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

## ANN from keras

In [29]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=56,activation='relu'),
    tf.keras.layers.Dense(units=32,activation='relu'),
    tf.keras.layers.Dense(units=16,activation='relu'),
    tf.keras.layers.Dense(units=10,activation='softmax')
])


In [30]:
model.compile(optimizer='adam',loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [34]:
model.fit(x_train,y_train1,epochs=20)

Epoch 1/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1143 - accuracy: 0.9646
Epoch 2/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0926 - accuracy: 0.9714
Epoch 3/20
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0723 - accuracy: 0.9771
Epoch 4/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0624 - accuracy: 0.9804
Epoch 5/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0522 - accuracy: 0.9826
Epoch 6/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0443 - accuracy: 0.9855
Epoch 7/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0402 - accuracy: 0.9870
Epoch 8/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0374 - accuracy: 0.9880
Epoch 9/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0343 - accuracy: 0.9886
Epoch 10/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.031

In [35]:
accuracy_score(y_test,np.argmax(model.predict(x_test),1))

313/313 [==============================] - 1s 3ms/step


0.9663

8